# Выбор локации для скважины

Нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. 

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Изучение данных</a></span></li></ul></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span><ul class="toc-item"><li><span><a href="#Готовим-признаки" data-toc-modified-id="Готовим-признаки-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Готовим признаки</a></span></li><li><span><a href="#Разбиваем-данные-на-выборки" data-toc-modified-id="Разбиваем-данные-на-выборки-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Разбиваем данные на выборки</a></span></li><li><span><a href="#Обучаем-и-проверяем-модель" data-toc-modified-id="Обучаем-и-проверяем-модель-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обучаем и проверяем модель</a></span></li></ul></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span><ul class="toc-item"><li><span><a href="#Считаем-прибыль" data-toc-modified-id="Считаем-прибыль-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Считаем прибыль</a></span></li><li><span><a href="#Считаем-распределение-прибыли-и-риски" data-toc-modified-id="Считаем-распределение-прибыли-и-риски-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Считаем распределение прибыли и риски</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li></ul></div>

## Загрузка и подготовка данных

In [1]:
# импорт магии
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.dummy import DummyRegressor

### Загрузка данных
Загрузим и сохраним три датафрейма с данными геологической разведки местрождений

In [2]:
try:
    geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
    geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
    geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    geo_data_0 = pd.read_csv('geo_data_0.csv')
    geo_data_1 = pd.read_csv('geo_data_1.csv')
    geo_data_2 = pd.read_csv('geo_data_2.csv')

### Изучение данных

In [3]:
# напишем функцию, которая нам все расскажет.
def exploring(df):
    display(df.head())
    display(df.info())
    display(df.describe())
    display('Пропуски: ', df.isna().mean())
    print('Количество строк-дубликатов:', df.duplicated().sum())
    print('--Корреляция--')
    display(df.corr())

In [4]:
# создадим список таблиц
data = [geo_data_0, geo_data_1, geo_data_2]
# применим функцию в цикле
for i in range(len(data)):
    print()
    print('--------------------------------------------')
    print('Информация по датафрейму региона', i+1)
    exploring(data[i])


--------------------------------------------
Информация по датафрейму региона 1


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


...

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


'Пропуски: '

id         0.0
f0         0.0
f1         0.0
f2         0.0
product    0.0
dtype: float64

Количество строк-дубликатов: 0
--Корреляция--


,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Проблемы с данными отсуствуют, не считая того, что корреляция между признаком `f2` и целевым признаком в `geo_data_1` равна единице. Будем считать, что я ослеп, ведь в задании написано, что данные синтетические.

Уберем ненужный нам признак из каждого датафрейма.

In [5]:
clean_geo_data_0 = geo_data_0.drop('id', axis=1)
clean_geo_data_1 = geo_data_1.drop('id', axis=1)
clean_geo_data_2 = geo_data_2.drop('id', axis=1)

## Обучение и проверка модели

### Готовим признаки
Разобьем на фичи и таргет

In [6]:
# напишем функцию по разбивке
def separation(df, trgt):
    features = df.drop(trgt, axis=1)
    target = df[trgt]
    return features, target

In [7]:
# применим ее для каждой таблицы и, для проверки, выведем размер
features_geo_0, target_geo_0 = separation(clean_geo_data_0, 'product')
display(features_geo_0.shape, target_geo_0.shape)
features_geo_1, target_geo_1 = separation(clean_geo_data_1, 'product')
display(features_geo_1.shape, target_geo_1.shape)
features_geo_2, target_geo_2 = separation(clean_geo_data_2, 'product')
display(features_geo_2.shape, target_geo_2.shape)

(100000, 3)

(100000,)

(100000, 3)

(100000,)

(100000, 3)

(100000,)

Поделено и проверено!

### Разбиваем данные на выборки
Делим на тренировочные и валидационные в соотношениии 3:1

In [8]:
# табличка раз
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_geo_0, 
                                                                              target_geo_0, 
                                                                              test_size=0.25, 
                                                                              random_state=42)
display(features_train_0.shape, 
        features_valid_0.shape, 
        target_train_0.shape, 
        target_valid_0.shape)

(75000, 3)

(25000, 3)

(75000,)

(25000,)

In [9]:
# табличка два
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_geo_1, 
                                                                              target_geo_1, 
                                                                              test_size=0.25, 
                                                                              random_state=42)
display(features_train_1.shape, 
        features_valid_1.shape, 
        target_train_1.shape, 
        target_valid_1.shape)

(75000, 3)

(25000, 3)

(75000,)

(25000,)

In [10]:
# табличка три
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_geo_2, 
                                                                              target_geo_2, 
                                                                              test_size=0.25, 
                                                                              random_state=42)
display(features_train_2.shape, 
        features_valid_2.shape, 
        target_train_2.shape, 
        target_valid_2.shape)

(75000, 3)

(25000, 3)

(75000,)

(25000,)

Все получилось

### Обучаем и проверяем модель

Создадим списки с фичами и таргетами, функцию для предсказания модели и цикл для вывода информации.

In [11]:
# Создадим списки фичей и таргетов для регионов
features_train = [features_train_0, features_train_1, features_train_2]
targets_train = [target_train_0, target_train_1, target_train_2]
features_valid = [features_valid_0, features_valid_1, features_valid_2]
targets_valid = [target_valid_0, target_valid_1, target_valid_2]

In [12]:
#создадим функцию, возвращающую prediction модели
def regressing(features_train, target_train, features_valid):
    lin_reg = LinearRegression()
    lin_reg.fit(features_train, target_train)
    predictions_valid = lin_reg.predict(features_valid)
    return predictions_valid

In [13]:
# создаем переменные с предсказаниями
predictions_valid_0 = regressing(features_train_0, target_train_0, features_valid_0)
predictions_valid_1 = regressing(features_train_1, target_train_1, features_valid_1)
predictions_valid_2 = regressing(features_train_2, target_train_2, features_valid_2)
# ну и список, конечно, для цикла
predictions_valid = [predictions_valid_0, predictions_valid_1, predictions_valid_2]

In [14]:
# прогоним циклом
for i in range(len(features_valid)):
    print('-----------------------------------------------')
    print('Регион', i + 1)
    print('Средний фактический запас нефти', targets_valid[i].mean(), 'баррелей')
    print('Средний предсказанный запас нефти', predictions_valid[i].mean(), 'баррелей')
    print('RMSE модели для данного региона', mean_squared_error(targets_valid[i], predictions_valid[i]) ** 0.5)

-----------------------------------------------
Регион 1
Средний фактический запас нефти 92.32595637084387 баррелей
Средний предсказанный запас нефти 92.39879990657768 баррелей
RMSE модели для данного региона 37.75660035026169
-----------------------------------------------
Регион 2
Средний фактический запас нефти 68.72538074722745 баррелей
Средний предсказанный запас нефти 68.71287803913762 баррелей
RMSE модели для данного региона 0.890280100102884
-----------------------------------------------
Регион 3
Средний фактический запас нефти 95.15099907171961 баррелей
Средний предсказанный запас нефти 94.77102387765939 баррелей
RMSE модели для данного региона 40.145872311342174


В регионе 2 очень маленький RMSE. Модель хорошо работает на данных из этого региона. В принципе, учитывая то, что мы видели ранее при построении корреляционной матрицы, по одному только признаку `f2` модель легко предскажет достоверный `product`, т.к. они абсолютно коррелируют. В общем, ничего неожиданного.

Можно еще, конечно, объединить все тренировочные фичи и таргеты по всем регионам в один датафрейм, так, возможно, даже правильнее, но я уже попробовал и получил для второго региона RMSE, равную ~39, что, конечно, не так хорошо, как сейчас.

Прежде, чем сделать выводы о качестве модели, сравним с дамми-моделью. Возьмем две с тратегии - среднюю и медиану. Они почти не отличаются, но разнообразие лучше однообразия.

In [15]:
# список стратегий
strategies = ['mean', 'median']

# наслаждаемся
for s in strategies:
    print()
    print('____________________')
    print('Дамми стратегия:', s)
    for i in range(len(features_valid)):
        dummy_reg = DummyRegressor(strategy=s)
        dummy_reg.fit(features_train[i], targets_train[i])
        predictions_dummy = dummy_reg.predict(features_valid[i])
        print('-----------------------------------------------')
        print('Регион', i + 1)
        print('Средний фактический запас нефти', targets_valid[i].mean(), 'баррелей')
        print('Средний предсказанный запас нефти', predictions_dummy.mean(), 'баррелей')
        print('RMSE модели для данного региона', mean_squared_error(targets_valid[i], predictions_dummy) ** 0.5)


____________________
Дамми стратегия: mean
-----------------------------------------------
Регион 1
Средний фактический запас нефти 92.32595637084387 баррелей
Средний предсказанный запас нефти 92.55801454305204 баррелей
RMSE модели для данного региона 44.27723501700235
-----------------------------------------------
Регион 2
Средний фактический запас нефти 68.72538074722745 баррелей
Средний предсказанный запас нефти 68.85820641759085 баррелей
RMSE модели для данного региона 45.94485941764291
-----------------------------------------------
Регион 3
Средний фактический запас нефти 95.15099907171961 баррелей
Средний предсказанный запас нефти 94.9496669760935 баррелей
RMSE модели для данного региона 44.78277680055413

____________________
Дамми стратегия: median
-----------------------------------------------
Регион 1
Средний фактический запас нефти 92.32595637084387 баррелей
Средний предсказанный запас нефти 92.18453272100078 баррелей
RMSE модели для данного региона 44.27685276083097
---

Наш результат превосходит результат дамми-моделей

## Подготовка к расчёту прибыли

Внесем данные для расчетов в переменные

In [16]:
BUDGET = 1.0E+10   # Общий бюджет на разработку
OILWELL_BUDGET = BUDGET / 200   # Бюджет на разработку одной скважины 
BARREL_PROFIT = 4.5E+5   # Доход от 1000 баррелей

Минимальный запас нефти в месторождении расчитывается как бюджет на одну точку, деленный на доход от 1000 баррелей.

In [17]:
min_oil_reserve = OILWELL_BUDGET / BARREL_PROFIT
min_oil_reserve

111.11111111111111

111 тыс. баррелей нефти минимум должно содержать каждое прибыльное месторождение. Это больше, чем средний запас нефти в месторождениях любого их рассматриваемых регионов. К счастью, в каждом регионе представленно по 100 тыс. точек нефтедобычи, а уж выбрать из них 200 самых прибыльных - задача посильная.

## Расчёт прибыли и рисков 

### Считаем прибыль

In [18]:
# переводим прогнозы из numpy в Series
predictions_valid_0 = pd.Series(predictions_valid_0)
predictions_valid_1 = pd.Series(predictions_valid_1)
predictions_valid_2 = pd.Series(predictions_valid_2)
# делаем список для цикла
predictions_valid = [predictions_valid_0, predictions_valid_1, predictions_valid_2]
# сбрасываем индекс у таргетов, чтобы он бился с прогнозами
target_valid_0 = target_valid_0.reset_index(drop=True)
target_valid_1 = target_valid_1.reset_index(drop=True)
target_valid_2 = target_valid_2.reset_index(drop=True)
# загоняем в список
targets_valid = [target_valid_0, target_valid_1, target_valid_2]

In [19]:
# создадим функцию расчета прибыли по реальным данным на основе предсказаний модели
def profit_calc(predictions, target):
    preds_sorted = (predictions
                    .sort_values(ascending=False))
    selected = target[preds_sorted.index][:200]
    return sum(selected) * BARREL_PROFIT - BUDGET

In [20]:
# посмотрим на результат
for i in range(len(predictions_valid)):
    print('----------------------------------------------')
    print('Прогнозируемая общая прибыль для лучших двухсот точек региона', i+1, 
          'равна', int(profit_calc(predictions_valid[i], targets_valid[i])), 'рублей')

----------------------------------------------
Прогнозируемая общая прибыль для лучших двухсот точек региона 1 равна 3359141114 рублей
----------------------------------------------
Прогнозируемая общая прибыль для лучших двухсот точек региона 2 равна 2415086696 рублей
----------------------------------------------
Прогнозируемая общая прибыль для лучших двухсот точек региона 3 равна 2598571759 рублей


Потенциально, конечно, прибыль приносят все регионы, а регион 1 больше всех. Но полезно посмотреть распределение прибыли.

### Считаем распределение прибыли и риски

Применим технику Bootstrap и оценим распределение прибыли. Найдем среднюю прибыль, 95% доверительный интервал, а заодно и поймем риски, остаться без прибыли. Сделаем 1000 выборок по 500 точек добычи и определим в каждой выборке 200 лучших, и их прибыль.

In [21]:
# настроим рандомайзер
state = np.random.RandomState(42)

In [22]:
# вложенные циклы наше все
for r in range(len(predictions_valid)):   # тут мы проходим по регионам
    values = []   # здесь будет прибыль топ-200 точек каждой выборки

    for i in range(1000):    # а тут генерим выборки
        target_subsample = targets_valid[r].sample(n=500, replace=True, random_state=state)
        preds_subsample = predictions_valid[r][target_subsample.index]
        values.append(profit_calc(preds_subsample, target_subsample))
    
    values = pd.Series(values)   # переводим на язык pandas
    lower = values.quantile(0.025)    # вычисляем левый край доверительного интервала
    higher = values.quantile(0.975)   # вычисляем правый край доверительного интервала
    print('Для региона', r+1, 'Средняя прибыль равна', int(values.mean()), 'рублей')
    print('95% процентный доверительный интервал', int(lower), '-', int(higher))
    print('Вероятность убытков в регионе', values[values < 0].count() / len(values)*100, '%')
    print('------------------------------------------------------------')

Для региона 1 Средняя прибыль равна 427847560 рублей
95% процентный доверительный интервал -97249829 - 954215192
Вероятность убытков в регионе 5.5 %
------------------------------------------------------------
Для региона 2 Средняя прибыль равна 511530218 рублей
95% процентный доверительный интервал 91700564 - 921455668
Вероятность убытков в регионе 0.6 %
------------------------------------------------------------
Для региона 3 Средняя прибыль равна 408545681 рублей
95% процентный доверительный интервал -120624872 - 960859440
Вероятность убытков в регионе 7.5 %
------------------------------------------------------------


### Вывод
Наиболее надежным с точки зрения вероятности получения прибыли является регион номер 2 (`geo_data_1`). 

Только в нем риск остаться без прибыли составляет менее 2.5% процентов. Кроме того, с  0.025 квантили на протяжении 95% доверительного интервала точки нефтедобычи региона 2 (`geo_data_1`) приносят прибыль. 